In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sweetviz

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import sweetviz as sv

In [6]:
cd '/content/drive/MyDrive/DCC/data'

/content/drive/MyDrive/DCC/data


In [34]:
train = pd.read_csv('./train.csv', index_col=0)
test = pd.read_csv('./test.csv', index_col=0)

In [ ]:
train.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
index,,,,,,,,,,,,,,,,,,,
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NAN,2.0,-6.0,1
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2


In [32]:
len(test)

20000

In [40]:
# 직업유형에서 결측치
# print(train.isnull().sum())
print(test.isnull().sum() )

child_num                                 10000
income_total                              10000
DAYS_BIRTH                                10000
DAYS_EMPLOYED                             10000
FLAG_MOBIL                                10000
work_phone                                10000
phone                                     10000
email                                     10000
family_size                               10000
begin_month                               10000
gender_F                                  10000
gender_M                                  10000
car_N                                     10000
car_Y                                     10000
reality_N                                 10000
reality_Y                                 10000
income_type_Commercial associate          10000
income_type_Pensioner                     10000
income_type_State servant                 10000
income_type_Student                       10000
income_type_Working                     

In [35]:
train.fillna('NAN', inplace=True)
test.fillna('NAN', inplace=True)

In [ ]:
advert_report = sv.analyze(train)
#display the report
advert_report.show_html('./sweetviz_Advertising2.html')

                                             |          | [  0%]   00:00 -> (? left)

Report ./sweetviz_Advertising2.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [11]:
#취업여부
train['emp'] = train['DAYS_EMPLOYED']>0
test['emp'] = test['DAYS_EMPLOYED']>0

In [12]:
# DAYS_BIRTH
train['DAYS_BIRTH_month']=np.floor((-train['DAYS_BIRTH'])/30)-((np.floor((-train['DAYS_BIRTH'])/30)/12).astype(int)*12)
train['DAYS_BIRTH_week']=np.floor((-train['DAYS_BIRTH'])/7)-((np.floor((-train['DAYS_BIRTH'])/7)/4).astype(int)*4)

# DAYS_EMPLOYED
train['DAYS_EMPLOYED_month']=np.floor((-train['DAYS_EMPLOYED'])/30)-((np.floor((-train['DAYS_EMPLOYED'])/30)/12).astype(int)*12)
train['DAYS_EMPLOYED_week']=np.floor((-train['DAYS_EMPLOYED'])/7)-((np.floor((-train['DAYS_EMPLOYED'])/7)/4).astype(int)*4)

# before_EMPLOYED
train['before_EMPLOYED']=train['DAYS_BIRTH']-train['DAYS_EMPLOYED']
train['before_EMPLOYED_month']=np.floor((-train['before_EMPLOYED'])/30)-((np.floor((-train['before_EMPLOYED'])/30)/12).astype(int)*12)
train['before_EMPLOYED_week']=np.floor((-train['before_EMPLOYED'])/7)-((np.floor((-train['before_EMPLOYED'])/7)/4).astype(int)*4)


# DAYS_BIRTH
test['DAYS_BIRTH_month']=np.floor((-test['DAYS_BIRTH'])/30)-((np.floor((-test['DAYS_BIRTH'])/30)/12).astype(int)*12)
test['DAYS_BIRTH_week']=np.floor((-test['DAYS_BIRTH'])/7)-((np.floor((-test['DAYS_BIRTH'])/7)/4).astype(int)*4)

# DAYS_EMPLOYED
test['DAYS_EMPLOYED_month']=np.floor((-test['DAYS_EMPLOYED'])/30)-((np.floor((-test['DAYS_EMPLOYED'])/30)/12).astype(int)*12)
test['DAYS_EMPLOYED_week']=np.floor((-test['DAYS_EMPLOYED'])/7)-((np.floor((-test['DAYS_EMPLOYED'])/7)/4).astype(int)*4)

# before_EMPLOYED
test['before_EMPLOYED']=test['DAYS_BIRTH']-test['DAYS_EMPLOYED']
test['before_EMPLOYED_month']=np.floor((-test['before_EMPLOYED'])/30)-((np.floor((-test['before_EMPLOYED'])/30)/12).astype(int)*12)
test['before_EMPLOYED_week']=np.floor((-test['before_EMPLOYED'])/7)-((np.floor((-test['before_EMPLOYED'])/7)/4).astype(int)*4)

In [14]:
train.loc[train['child_num'] >= 6,'child_num']=6
train.loc[train['family_size'] >= 8,'family_size']=8

test.loc[test['child_num'] >= 6,'child_num']=6
test.loc[test['family_size'] >= 8,'family_size']=8

In [ ]:
print(train.dtypes)

gender                    object
car                       object
reality                   object
child_num                  int64
income_total             float64
income_type               object
edu_type                  object
family_type               object
house_type                object
DAYS_BIRTH                 int64
DAYS_EMPLOYED              int64
FLAG_MOBIL                 int64
work_phone                 int64
phone                      int64
email                      int64
occyp_type                object
family_size              float64
begin_month              float64
credit                    object
emp                         bool
DAYS_BIRTH_month         float64
DAYS_BIRTH_week          float64
DAYS_EMPLOYED_month      float64
DAYS_EMPLOYED_week       float64
before_EMPLOYED            int64
before_EMPLOYED_month    float64
before_EMPLOYED_week     float64
dtype: object


In [36]:
object_col = []
object_col2 = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)
for col in train.columns:
    if train[col].dtype == 'object':
        object_col2.append(col)

In [37]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

In [38]:
test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col2]).toarray(), 
             columns=enc.get_feature_names(object_col2))
test.drop(object_col2, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)

In [ ]:
train=train.drop(['FLAG_MOBIL'], axis=1)
test=test.drop(['FLAG_MOBIL'], axis=1)

In [ ]:
train = train.astype({'credit': 'object'})
print(train.dtypes)

child_num                            int64
income_total                       float64
DAYS_BIRTH                           int64
DAYS_EMPLOYED                        int64
FLAG_MOBIL                           int64
                                    ...   
occyp_type_Realty agents           float64
occyp_type_Sales staff             float64
occyp_type_Secretaries             float64
occyp_type_Security staff          float64
occyp_type_Waiters/barmen staff    float64
Length: 65, dtype: object


In [ ]:
train.head()

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit,emp,DAYS_BIRTH_month,DAYS_BIRTH_week,DAYS_EMPLOYED_month,DAYS_EMPLOYED_week,before_EMPLOYED,before_EMPLOYED_month,before_EMPLOYED_week,gender_F,gender_M,car_N,car_Y,reality_N,reality_Y,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_NAN,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff
0,0,202500.0,-13899,-4709,1,0,0,0,2.0,-6.0,1,False,7.0,1.0,0.0,0.0,-9190,6.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,247500.0,-11380,-1540,1,0,0,1,3.0,-5.0,1,False,7.0,1.0,3.0,0.0,-9840,4.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,450000.0,-19087,-4434,1,0,1,0,2.0,-22.0,2,False,0.0,2.0,3.0,1.0,-14653,8.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,202500.0,-15088,-2092,1,0,1,0,2.0,-37.0,0,False,10.0,3.0,9.0,2.0,-12996,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,157500.0,-15037,-2105,1,0,0,0,2.0,-26.0,2,False,9.0,0.0,10.0,0.0,-12932,11.0,3.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train.to_csv('train_n.csv', index=False)
test.to_csv('test_n.csv', index=False)